In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../00-data/data.csv', encoding='latin1')

Proverò a studiare il dataset e trovare variabili categoriche nominali e ordinali

In [3]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


InvoiceNo: Variabile Categorica nominale forse univoca, se non univoca potrebbe essere interessante per ricostruire fatture
StockCode: Variabile Categorica nominale, potrebbe avere una relazione 1->1 con description, potrebbe essere interessante per raggruppamenti
Description: Variabile Categorica nominale, se 1->1 con StockCode potrebbe aver senso toglierla
Quantity: Variabile numerica
InvoiceDate: Variabile Categorica Ordinata
UnitPrice: Variabile Numerica
CustomerID: Variabile Categorica Nominale, potrebbe essere interessante per raggruppamenti
Country: Variabile Categorica Nominale, potrebbe essere interessante per raggruppamenti

In [5]:
df.shape

(541909, 8)

In [6]:
df['InvoiceNo'].unique()

array(['536365', '536366', '536367', ..., '581585', '581586', '581587'],
      shape=(25900,), dtype=object)

in questo esempio abbiamo 25900 fatture a fronte di 541909 entries, quindi InvoiceNo rappresenta una riga fattura

In [7]:
(df['StockCode'].unique().shape, df['Description'].unique().shape)

((4070,), (4224,))

Il numero di descrizioni sembra essere maggiore degli stockcode, forse è cambiata nel tempo

In [8]:
df_desc_unique = df.copy()

df_frequencies = df_desc_unique.groupby('StockCode')['Description'].agg('nunique').sort_values(ascending=False)

df_frequencies

StockCode
20713     8
23084     7
21830     6
85175     6
21181     5
         ..
20738     0
20689     0
17011A    0
16053     0
10123G    0
Name: Description, Length: 4070, dtype: int64

In [9]:
df_frequencies.info()

<class 'pandas.core.series.Series'>
Index: 4070 entries, 20713 to 10123G
Series name: Description
Non-Null Count  Dtype
--------------  -----
4070 non-null   int64
dtypes: int64(1)
memory usage: 192.6+ KB


In [10]:
df_frequencies.value_counts()

Description
1    3308
2     517
0     112
3      98
4      26
5       5
6       2
8       1
7       1
Name: count, dtype: int64

verosimilmente i le description sono agganciate allo stockcode, ma ogni tanto vengono cambiate, quindi ai fini pratici posso rimuovere le colonne description, InvoiceNo, StockCode, CustomerID

In [11]:
df_copy = df.copy()

In [12]:
df_copy = df_copy.drop(columns=["Description", 'InvoiceNo', 'StockCode', 'CustomerID',])

df_copy

,Quantity,InvoiceDate,UnitPrice,Country
0,6,12/1/2010 8:26,2.55,United Kingdom
1,6,12/1/2010 8:26,3.39,United Kingdom
2,8,12/1/2010 8:26,2.75,United Kingdom
3,6,12/1/2010 8:26,3.39,United Kingdom
4,6,12/1/2010 8:26,3.39,United Kingdom
...,...,...,...,...
541904,12,12/9/2011 12:50,0.85,France
541905,6,12/9/2011 12:50,2.10,France
541906,4,12/9/2011 12:50,4.15,France
541907,4,12/9/2011 12:50,4.15,France


In [13]:
df_copy.columns

Index(['Quantity', 'InvoiceDate', 'UnitPrice', 'Country'], dtype='object')

al campo  'Country' sarà applicata la one hot encoding

andiamo con pandas

In [14]:
#vai con la one hot encoding via pandas
df_copy = pd.get_dummies(df_copy, columns=['Country'], dtype="int")


In [15]:
df_copy

,Quantity,InvoiceDate,UnitPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,...,Country_RSA,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified
0,6,12/1/2010 8:26,2.55,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,8,12/1/2010 8:26,2.75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,12,12/9/2011 12:50,0.85,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541905,6,12/9/2011 12:50,2.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541906,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541907,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ora proviamo con il label encoding

aggiungo un parametro nella spesa che mostra il totale

In [16]:
df_copy['totale'] = df_copy['UnitPrice'] * df_copy['Quantity']
df_copy

,Quantity,InvoiceDate,UnitPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,...,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,totale
0,6,12/1/2010 8:26,2.55,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,15.30
1,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,20.34
2,8,12/1/2010 8:26,2.75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,22.00
3,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,20.34
4,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,12,12/9/2011 12:50,0.85,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10.20
541905,6,12/9/2011 12:50,2.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12.60
541906,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16.60
541907,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16.60


vedo i dati statistici a riguardo per trovare 3 treshold

In [17]:
df_copy['totale'].describe()

count    541909.000000
mean         17.987795
std         378.810824
min     -168469.600000
25%           3.400000
50%           9.750000
75%          17.400000
max      168469.600000
Name: totale, dtype: float64

In [18]:
df_copy.sort_values()

TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

In [ ]:
import numpy as np

In [ ]:
bins = [-np.inf, -0.001, 0.001, 9.999, 99.999, np.inf]
labels = ['restitution', 'free', 'small', 'medium', 'big']

dimension_acquire = pd.cut(df_copy['totale'], bins=bins, labels=labels, right=True)

df_copy['DimensionAcuqire'] = dimension_acquire

df_copy

,Quantity,InvoiceDate,UnitPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,totale,DimensionAcuqire
0,6,12/1/2010 8:26,2.55,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,15.30,medium
1,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,medium
2,8,12/1/2010 8:26,2.75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,22.00,medium
3,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,medium
4,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,12,12/9/2011 12:50,0.85,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10.20,medium
541905,6,12/9/2011 12:50,2.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12.60,medium
541906,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16.60,medium
541907,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16.60,medium


In [ ]:
df_copy['DimensionAcuqire'].unique()



['medium', 'small', 'big', 'restitution', 'free']
Categories (5, object): ['restitution' < 'free' < 'small' < 'medium' < 'big']

ora vado a trasformarlo in variabili numeriche

In [ ]:
remap_dimension_acquire = {'restitution': 1, 'free': 2, 'small': 3, 'medium': 4, 'big': 5}

remap_dimension_acquire

{'restitution': 1, 'free': 2, 'small': 3, 'medium': 4, 'big': 5}

In [ ]:
df_copy['DimensionAcuqire'] = df_copy['DimensionAcuqire'].map(remap_dimension_acquire)

df_copy

,Quantity,InvoiceDate,UnitPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,totale,DimensionAcuqire
0,6,12/1/2010 8:26,2.55,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,15.30,4
1,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,4
2,8,12/1/2010 8:26,2.75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,22.00,4
3,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,4
4,6,12/1/2010 8:26,3.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,20.34,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,12,12/9/2011 12:50,0.85,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10.20,4
541905,6,12/9/2011 12:50,2.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12.60,4
541906,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16.60,4
541907,4,12/9/2011 12:50,4.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16.60,4


In [ ]:
df_copy['DimensionAcuqire'].unique()


[4, 3, 5, 1, 2]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

da qui continuerò aggiungendo anche le versioni basate su numpy

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


In [ ]:
X = df.values
X

array([['536365', '85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', ...,
        2.55, 17850.0, 'United Kingdom'],
       ['536365', '71053', 'WHITE METAL LANTERN', ..., 3.39, 17850.0,
        'United Kingdom'],
       ['536365', '84406B', 'CREAM CUPID HEARTS COAT HANGER', ..., 2.75,
        17850.0, 'United Kingdom'],
       ...,
       ['581587', '23254', 'CHILDRENS CUTLERY DOLLY GIRL ', ..., 4.15,
        12680.0, 'France'],
       ['581587', '23255', 'CHILDRENS CUTLERY CIRCUS PARADE', ..., 4.15,
        12680.0, 'France'],
       ['581587', '22138', 'BAKING SET 9 PIECE RETROSPOT ', ..., 4.95,
        12680.0, 'France']], shape=(541909, 8), dtype=object)

In [ ]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


rimuovo le colonne description, InvoiceNo, StockCode, CustomerID

In [ ]:
X = np.delete(X, [0,1,2,6], axis=1)

X

array([[6, '12/1/2010 8:26', 2.55, 'United Kingdom'],
       [6, '12/1/2010 8:26', 3.39, 'United Kingdom'],
       [8, '12/1/2010 8:26', 2.75, 'United Kingdom'],
       ...,
       [4, '12/9/2011 12:50', 4.15, 'France'],
       [4, '12/9/2011 12:50', 4.15, 'France'],
       [3, '12/9/2011 12:50', 4.95, 'France']],
      shape=(541909, 4), dtype=object)

ora mappiamo le categorie nominali

In [ ]:
# enc = OneHotEncoder()
# X_enc = enc.fit_transform(X, categorical_features=[3])

# X_enc

ct = ColumnTransformer(
    transformers=[
        # ('num', StandardScaler(), [0, 1]),
        ('cat', OneHotEncoder(), [3])
    ],
    remainder='drop'
)

X_enc = ct.fit_transform(X)

X_enc

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 541909 stored elements and shape (541909, 38)>

In [ ]:
print(X_enc)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 541909 stored elements and shape (541909, 38)>
  Coords	Values
  (0, 36)	1.0
  (1, 36)	1.0
  (2, 36)	1.0
  (3, 36)	1.0
  (4, 36)	1.0
  (5, 36)	1.0
  (6, 36)	1.0
  (7, 36)	1.0
  (8, 36)	1.0
  (9, 36)	1.0
  (10, 36)	1.0
  (11, 36)	1.0
  (12, 36)	1.0
  (13, 36)	1.0
  (14, 36)	1.0
  (15, 36)	1.0
  (16, 36)	1.0
  (17, 36)	1.0
  (18, 36)	1.0
  (19, 36)	1.0
  (20, 36)	1.0
  (21, 36)	1.0
  (22, 36)	1.0
  (23, 36)	1.0
  (24, 36)	1.0
  :	:
  (541884, 36)	1.0
  (541885, 36)	1.0
  (541886, 36)	1.0
  (541887, 36)	1.0
  (541888, 36)	1.0
  (541889, 36)	1.0
  (541890, 36)	1.0
  (541891, 36)	1.0
  (541892, 36)	1.0
  (541893, 36)	1.0
  (541894, 13)	1.0
  (541895, 13)	1.0
  (541896, 13)	1.0
  (541897, 13)	1.0
  (541898, 13)	1.0
  (541899, 13)	1.0
  (541900, 13)	1.0
  (541901, 13)	1.0
  (541902, 13)	1.0
  (541903, 13)	1.0
  (541904, 13)	1.0
  (541905, 13)	1.0
  (541906, 13)	1.0
  (541907, 13)	1.0
  (541908, 13)	1.0
